In [34]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import *
from keras import initializers

In [38]:
#Clean the cosmetic data
cos_df = pd.read_csv('data/Cosmetic_products_sales.csv')

cos_df = cos_df.drop(columns=['Unit',"Pack Unit Id",'Master Category','Size','Rank','Date','Amount to Customer']).dropna()

default = [4,2017]
current = default.copy()

missing_times = []
last = ''
past = 0

counter = 0
# Iterate through each row in the dataframe
for index, row in cos_df.iterrows():
    curr = row['Site Id']
    month = row['Month']
    year = row['Year']

    counter += 1
    if curr != last:
        current = default.copy()
        past = row["Net Sales calculated"]
        last = curr

    while month != current[0] or year != current[1]:
        dup = row.copy()
        dup['Month'] = current[0]
        dup['Year'] = current[1]
        
        missing_times.append(dup.tolist())
        current[0] += 1
        if current[0] > 12:
            current[0] = 1
            current[1] += 1
    current[0] += 1
    if current[0] > 12:
        current[0] = 1
        current[1] += 1
    # cos_df.loc[index, "Net Sales calculated"] -= past

missing_df = pd.DataFrame(missing_times, columns=cos_df.columns)
cos_df = pd.concat([cos_df, missing_df], ignore_index=True)

cos_df['Date'] = cos_df['Year'].astype(str) + '-' + cos_df['Month'].astype(str)

# Convert the combined column to datetime
cos_df['Date'] = pd.to_datetime(cos_df['Date'])

# Drop the original year and month columns if needed
cos_df = cos_df.drop(columns=['Year', 'Month'])

string_encoded = ['ParentSKU', 'Site Id','State','Zone']
# normalize = ["Net Sales calculated","Cash Discount","Amount to Customer","Qty","Price","MRP","Pack Size"]
cos_df_sales = cos_df["Net Sales calculated"]

for i in cos_df.columns:
    if i in string_encoded:
        cos_df[i] = pd.factorize(cos_df[i])[0]
        
cos_df = cos_df.sort_values(by=['ParentSKU','Site Id','Date']).reset_index(drop=True)
cos_df

,ParentSKU,Site Id,Category Name ID,Qty,Price,Net Sales calculated,Cash Discount,MRP,Pack Size,State,Zone,Date
0,0,0,0,1621.0,54,82557,20,80.0,30.0,0,0,2017-04-01
1,0,0,0,651.0,51,31473,21,80.0,30.0,0,0,2017-05-01
2,0,0,0,457.0,46,19580,12,70.0,30.0,0,0,2017-06-01
3,0,0,0,1985.0,41,78144,0,70.0,30.0,0,0,2017-07-01
4,0,0,0,6.0,38,228,0,70.0,30.0,0,0,2017-08-01
...,...,...,...,...,...,...,...,...,...,...,...,...
42206,51,24,1,1179.0,40,44215,0,60.0,120.0,0,3,2019-10-01
42207,51,24,1,81.0,40,2947,0,60.0,120.0,0,3,2019-11-01
42208,51,24,1,81.0,40,2947,0,60.0,120.0,0,3,2019-12-01
42209,51,24,1,580.0,40,21125,0,60.0,120.0,0,3,2020-01-01


In [18]:
shampoo_df = pd.read_csv('data/shampoo_sales.csv')
shampoo_df = shampoo_df['Sales']
shampoo_answers = shampoo_df
shampoo_df = shampoo_df.apply(lambda x: (x - shampoo_df.min()) / (shampoo_df.max() - shampoo_df.min()))

In [19]:
def transform_shampoo_data(shampoo_df,shampoo_answers,window_size=5):
    shampoo_np = shampoo_df.to_numpy()
    x = []
    y = []
    for i in range(len(shampoo_np)-window_size):
        row = [[j] for j in shampoo_np[i:i+window_size].astype(float)]    
        x.append(row)
        y.append(shampoo_answers[i+window_size].astype(float))
    
    return np.array(x), np.array(y)
shampoo_x, shampoo_y = transform_shampoo_data(shampoo_df,shampoo_answers,6)
shampoo_x


array([[[0.2607073 ],
        [0.04727208],
        [0.11338191],
        [0.        ],
        [0.1084059 ],
        [0.08743558]],

       [[0.04727208],
        [0.11338191],
        [0.        ],
        [0.1084059 ],
        [0.08743558],
        [0.19992891]],

       [[0.11338191],
        [0.        ],
        [0.1084059 ],
        [0.08743558],
        [0.19992891],
        [0.18695575]],

       [[0.        ],
        [0.1084059 ],
        [0.08743558],
        [0.19992891],
        [0.18695575],
        [0.13062022]],

       [[0.1084059 ],
        [0.08743558],
        [0.19992891],
        [0.18695575],
        [0.13062022],
        [0.00639773]],

       [[0.08743558],
        [0.19992891],
        [0.18695575],
        [0.13062022],
        [0.00639773],
        [0.38599609]],

       [[0.19992891],
        [0.18695575],
        [0.13062022],
        [0.00639773],
        [0.38599609],
        [0.11835792]],

       [[0.18695575],
        [0.13062022],
        [0.0063977

In [20]:
limit = 25
shampoo_train_x, shampoo_train_y = shampoo_x[:limit],shampoo_y[:limit]
shampoo_test_x, shampoo_test_y = shampoo_x[limit:],shampoo_y[limit:]


In [21]:
model = Sequential()
model.add(LSTM(units=64, input_shape=(shampoo_train_x.shape[1], shampoo_train_x.shape[2])))
model.add(Dense(4, 'relu'))
model.add(Dense(1, 'linear'))
model.compile(loss='mae', optimizer=Adam(learning_rate=0.0000001), metrics=[RootMeanSquaredError()])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [22]:
model.fit(shampoo_train_x, shampoo_train_y, epochs=10, batch_size=4)

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 267.3059 - root_mean_squared_error: 284.3565  
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 290.1727 - root_mean_squared_error: 302.0149
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 293.9847 - root_mean_squared_error: 307.8987
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 294.1776 - root_mean_squared_error: 313.8052
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 285.4972 - root_mean_squared_error: 301.7723
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 276.6686 - root_mean_squared_error: 292.6544
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 302.6716 - root_mean_squared_error: 321.9767 
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 314.5940 - root_mean_squared_error: 336.5992
Epoch 9/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 303.2765 - root_mean_squared_error: 313.3513 
Epoch 10/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 294.0514 - root_mean_squ

In [23]:
train_predictions = model.predict(shampoo_test_x).flatten()
train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':shampoo_test_y})
train_results

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step


,Train Predictions,Actuals
0,0.010406,407.6
1,0.009891,682.0
2,0.012058,475.3
3,0.011677,581.3
4,0.011980,646.9


In [24]:
cos_df

,ParentSKU,Site Id,Year,Month,Category Name ID,Qty,Price,Net Sales calculated,Cash Discount,MRP,Pack Size,State,Zone
0,0,0,0,0,0,0.031020,0.027108,0.015148,0.061538,0.031480,0.041488,0,0
1,0,0,0,1,0,0.012446,0.024096,0.005772,0.064615,0.031480,0.041488,0,0
2,0,0,0,2,0,0.008731,0.019076,0.003589,0.036923,0.024659,0.041488,0,0
3,0,0,0,3,0,0.037989,0.014056,0.014338,0.000000,0.024659,0.041488,0,0
4,0,0,0,4,0,0.000096,0.011044,0.000037,0.000000,0.024659,0.041488,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42206,51,24,2,6,1,0.022556,0.013052,0.008110,0.000000,0.017838,0.170243,0,3
42207,51,24,2,7,1,0.001532,0.013052,0.000536,0.000000,0.017838,0.170243,0,3
42208,51,24,2,8,1,0.001532,0.013052,0.000536,0.000000,0.017838,0.170243,0,3
42209,51,24,3,9,1,0.011087,0.013052,0.003872,0.000000,0.017838,0.170243,0,3


In [25]:
def timeseries_cos_data(cos_df,cos_answers,window_size=6,sample_size=33):
    cos_np = cos_df.astype(float).to_numpy()   
    x = []
    y = []
    for i in range(len(cos_np)//sample_size):
        for j in range(sample_size-window_size):
            row = [k for k in cos_np[i*sample_size+j:i*sample_size+j+window_size]]    
            x.append(row)
            y.append(cos_answers[i*sample_size+j+window_size].astype(float))
    return np.array(x), np.array(y)
cos_x, cos_y = timeseries_cos_data(cos_df,cos_df_sales,6)
cos_y

array([ 64639.,  64639., 136603., ...,   8667.,  38321.,  44215.])

In [26]:
limit = 30000

cos_train_x, cos_train_y = cos_x[:limit],cos_y[:limit]
cos_test_x, cos_test_y = cos_x[limit:],cos_y[limit:]
cos_train_x.shape, cos_train_y.shape

((30000, 6, 13), (30000,))

In [27]:
model4 = Sequential()
model4.add(InputLayer((cos_train_x.shape[1], cos_train_x.shape[2])))  # Input shape (time_steps, features)
model4.add(LSTM(800, return_sequences=True))  # First LSTM layer, return_sequences=True to pass output to next LSTM layer
model4.add(LSTM(400, return_sequences=True))  # First LSTM layer, return_sequences=True to pass output to next LSTM layer
model4.add(LSTM(200, return_sequences=True))  # First LSTM layer, return_sequences=True to pass output to next LSTM layer
model4.add(LSTM(100, return_sequences=True))  # Second LSTM layer
model4.add(LSTM(50))  # Second LSTM layer
# model4.add(Dense(8, activation='relu'))
model4.add(Dense(1))

model4.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_7 (LSTM)                   │ (None, 6, 800)         │     2,604,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 6, 400)         │     1,921,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 6, 200)         │       480,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 6, 100)         │       120,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 50)             │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,157,851 (19.68 MB)

 Trainable params: 5,157,851 (19.68 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
cp4 = ModelCheckpoint('data/model4/model4.keras', save_best_only=True)
model4.compile(loss='mae', optimizer=Adam(learning_rate=0.0001))
model4.fit(cos_train_x, cos_train_y, validation_data=(cos_test_x, cos_test_y),callbacks=[cp4], epochs=10,batch_size=64)

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 14s 25ms/step - loss: 66590.0078 - val_loss: 188725.1250
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - loss: 67006.5938 - val_loss: 188722.3750
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - loss: 65727.4219 - val_loss: 188719.9844
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - loss: 66772.3750 - val_loss: 188717.5312
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - loss: 67226.1094 - val_loss: 188715.1250
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - loss: 65904.5156 - val_loss: 188712.7031
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - loss: 66745.0859 - val_loss: 188710.2656
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - loss: 66811.6875 - val_loss: 188707.8906
Epoch 9/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - loss: 65317.4922 - val_loss: 188705.5000
Epoch 10/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - loss: 65693.1328 - val_loss: 188703.1562


In [29]:
cos_predictions = model4.predict(cos_train_x).flatten()
train_results = pd.DataFrame(data={'Train Predictions':cos_predictions, 'Actuals':cos_train_y})
train_results

938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step


,Train Predictions,Actuals
0,34.639301,64639.0
1,34.655674,64639.0
2,34.661789,136603.0
3,34.663841,68425.0
4,34.664791,258791.0
...,...,...
29995,34.668720,368880.0
29996,34.668720,368880.0
29997,34.668720,255542.0
29998,34.668720,170814.0


In [30]:
cos_train_y

array([ 64639.,  64639., 136603., ..., 255542., 170814., 118116.])

In [31]:
cos_test_y

array([402769., 350293.,  84116., ...,   8667.,  38321.,  44215.])

In [32]:
cos_train_x

array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         4.14878398e-02, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         4.14878398e-02, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         4.14878398e-02, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         4.14878398e-02, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         4.14878398e-02, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         4.14878398e-02, 0.00000000e+00, 0.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         4.14878398e-02, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         4.14878398e-02, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+0